## Imports

In [1]:
%%time
'''
import pandas  as pd
import seaborn as sns
import numpy   as np
import matplotlib.pyplot as plt
from scipy.signal    import find_peaks
from scipy.integrate import simps, trapz
'''

from mainFile import *

mainFile e bibliotecas importados
Wall time: 2.61 s


# Lendo arquivo das waveforms

#### O arquivo original já foi parcialmente modificado em relação a linhas e colunas

In [2]:
%%time
'''Lendo arquivo das waveforms'''

waveform = pd.read_csv(
    'data/5555_eventos-edit.csv', 
    index_col = 0
                      ) # importa como waveform vs sample
# waveform

Wall time: 2.85 s


In [3]:
#plt.plot( waveform.time, waveform.event_0 )
#waveform.shape[1] - 1

# 3. Análise de dados

## 3.1. Copia o data frame e seleciona a janela de busca

Cria uma cópia das waveforms para poder transladar o base line sem modificar os dados originais;

Define a janela onde procuraremos pelos valores do contorno do pulso

In [4]:
df = waveform.copy(deep = True)
df = df.iloc[ : , :df.shape[1] - 1 ]

VA_1 = 20 # VA_1 e VA_2 são variáveis arbitrárias para definir o majorante da largura dos pulsos
VA_2 = 70

convert_to_microsec = 1/250

df.head()

,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,...,event_5545,event_5546,event_5547,event_5548,event_5549,event_5550,event_5551,event_5552,event_5553,event_5554
0,56.0,56.0,56.0,53.0,55.0,55.0,54.0,54.0,54.0,54.0,...,54.0,54.0,53.0,53.0,51.0,51.0,51.0,51.0,51.0,51.0
1,55.0,55.0,55.0,53.0,55.0,55.0,54.0,54.0,54.0,54.0,...,52.0,52.0,52.0,52.0,52.0,51.0,51.0,51.0,51.0,51.0
2,53.0,53.0,53.0,51.0,53.0,53.0,55.0,55.0,55.0,55.0,...,53.0,53.0,53.0,53.0,50.0,55.0,55.0,55.0,55.0,55.0
3,53.0,53.0,53.0,52.0,53.0,53.0,54.0,54.0,54.0,54.0,...,53.0,53.0,53.0,53.0,50.0,55.0,55.0,55.0,55.0,55.0
4,54.0,54.0,54.0,53.0,55.0,55.0,55.0,55.0,55.0,55.0,...,51.0,51.0,53.0,53.0,52.0,54.0,54.0,54.0,54.0,54.0


## 3.2. Translação da base-line

É interessante transladar os pontos e fixar a base line no 0. Podemos fazer isso somando ~= -50 em cada um dos valores de toda a coleção de waveforms. Isso servirá para evitar problemas de 'height' no peak_finder()

Análise estatística do base-line:
No caso, tomaremos todos os dados antes do primeiro pulso de cada waveform e consideraremos o base-line como o mesmo para todos os waveforms

#### Método:

Para encontrar a base line, selecionaremos a parte mais à esquerda, antes do pulso. 
Esse pedaço é definido como fora de um valor arbitrário em torno do primeiro pico da waveform, que acaba não sofrendo com a questão da base line. Definimos desse jeito porque a função para calcular o contorno do pulso, além de demorar para ser computada, sofre com as questões do peak_finder acabar encontrando apenas um elemento


In [5]:
%%time

height = 50 # esse parâmetro aqui é somente para efeito de encontrar os elementos da  base line a priori

baseLines = baseLine(df, height = height, VA_1 = VA_1)

Wall time: 1.76 s


In [6]:
# baseLines

In [7]:
# baseLines.value_counts()

In [8]:
# baseLines.mean() , baseLines.std()

#### Relocação do base-line
Colocando o valor médio do base line como o parâmetro height do peak-finder, surpreendentemente temos um resultado que não incorre em erros na hora de achar os dois picos de cada waveform. No caso, ele acaba por achar dois picos (não mais nem menos) de maneira mais fácil e exata

In [9]:
'''
Alterando os valores e recolocando na base line ~= 0
'''

df.iloc[ :  , : df.shape[1] ] -= baseLines.mean() 
# soma ~= -50 em todo o df, exceto pela última coluna 'time'
df.head()

,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,...,event_5545,event_5546,event_5547,event_5548,event_5549,event_5550,event_5551,event_5552,event_5553,event_5554
0,3.72321,3.72321,3.72321,0.72321,2.72321,2.72321,1.72321,1.72321,1.72321,1.72321,...,1.72321,1.72321,0.72321,0.72321,-1.27679,-1.27679,-1.27679,-1.27679,-1.27679,-1.27679
1,2.72321,2.72321,2.72321,0.72321,2.72321,2.72321,1.72321,1.72321,1.72321,1.72321,...,-0.27679,-0.27679,-0.27679,-0.27679,-0.27679,-1.27679,-1.27679,-1.27679,-1.27679,-1.27679
2,0.72321,0.72321,0.72321,-1.27679,0.72321,0.72321,2.72321,2.72321,2.72321,2.72321,...,0.72321,0.72321,0.72321,0.72321,-2.27679,2.72321,2.72321,2.72321,2.72321,2.72321
3,0.72321,0.72321,0.72321,-0.27679,0.72321,0.72321,1.72321,1.72321,1.72321,1.72321,...,0.72321,0.72321,0.72321,0.72321,-2.27679,2.72321,2.72321,2.72321,2.72321,2.72321
4,1.72321,1.72321,1.72321,0.72321,2.72321,2.72321,2.72321,2.72321,2.72321,2.72321,...,-1.27679,-1.27679,0.72321,0.72321,-0.27679,1.72321,1.72321,1.72321,1.72321,1.72321
